cwd -> Root Directory

In [1]:
import os
os.chdir('../')
%pwd

'e:\\code\\project\\Text-Summarizer'

Creating Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path

    # Parameters
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

Configuration Manager

In [3]:
from textSummarizer.utils import read_yaml,create_directories 
from textSummarizer import constants

class ConfigurationManager:      #class to read the config file and params file
    def __init__(
            self,
            config_file_path = constants.CONFIG_FILE_PATH,       #constants is a file which contains all the paths 
            params_file_path = constants.PARAMS_FILE_PATH
            ):
         
        self.config = read_yaml(config_file_path)      #read_yaml is a function which reads the yaml file and returns a dictionary
        self.params = read_yaml(params_file_path)

        #config is a ConfigBox(shown in trials.ipynb) object which is a dictionary with dot notation access with help of read_yaml function
        create_directories([self.config.artifacts_root])              #create_directories is a function which creates directories if they don't exist; artifacts_root is the path where all the artifacts will be stored present in the config.yaml file


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_training
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,

            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )


Creating Components

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'            #check if gpu is available or not and sets the device accordingly
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)         #load the tokenizer 
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)        #load the model and send it to the device (gpu or cpu)
        seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)             #data collector for seq2seq model which will be used for training the model 

        dataset_samsum_pt = load_from_disk(self.config.data_path)           #load the dataset from the disk 

        
        trainer_args = TrainingArguments(                   #training arguments for the trainer present in yaml file
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )


        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collector,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
            )
        
        trainer.train()             #train the model

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-model'))           #save the model in the root directory 

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-tokenizer'))           #save the tokenizer in the root directory        
    

d:\anaconda\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating Pipeline

In [ ]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e